### Q1.Design a Sudoku puzzle where the board consists of 81 squares, some of which are initially filled with digits from 1 to 9.  

The puzzle is to fill in all the remaining squares such that no digit appears twice in any row, column, or 3 × 3 box.  
A row, column, or box is called a unit.  

1.	Represent the Sudoku problem in  CSP by identifying the variable, domain, and constraint.
2.	Implement the problem using backtracking search, what is avg. time taken by the algorithm for 10 runs.
3.	Analyse how different fault finding algorithms such as Forward Checking, Arc consistency improve the computational time of backtracking search?
4.	Analyse how different Heuristics MRV (Minimum Remaining Values), Degree heuristic, Least Constraining Value affect the  computational time of backtracking search?


In [1]:
import numpy as np
from pprint  import pprint 
import sys,warnings,copy, random
warnings.filterwarnings("ignore", category=UserWarning)  
random.seed(42)
np.random.seed(42)

In [2]:
base  = 3
side  = base*base

# pattern for a baseline valid solution
def pattern(r,c): return (base*(r%base)+r//base+c)%side

# randomize rows, columns and numbers (of valid base pattern)
from random import sample
def shuffle(s): return sample(s,len(s)) 
rBase = range(base) 
rows  = [ g*base + r for g in shuffle(rBase) for r in shuffle(rBase) ] 
cols  = [ g*base + c for g in shuffle(rBase) for c in shuffle(rBase) ]
nums  = shuffle(range(1,base*base+1))

# produce board using randomized baseline pattern
solution = [ [nums[pattern(r,c)] for c in cols] for r in rows ]
# print(solution)
board = np.array(solution)

# original_array = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# # Deep copy of the array
# copied_array = original_array.copy()

squares = side*side
empties = squares * 3//4
for p in sample(range(squares),empties):
    board[p//side][p%side] = 0

numSize = len(str(side))
# for line in board:
#     print(*(f"{n or '.':{numSize}} " for n in line))
# print(solution)

In [3]:
def shortSudokuSolve(board):
    side   = len(board)
    base   = int(side**0.5)
    board  = [n for row in board for n in row ]
    blanks = [i for i,n in enumerate(board) if n==0 ]
    cover  = { (n,p):{*zip([2*side+r, side+c, r//base*base+c//base],[n]*(n and 3))}
                for p in range(side*side) for r,c in [divmod(p,side)] for n in range(side+1) }
    used   = set().union(*(cover[n,p] for p,n in enumerate(board) if n))
    placed = 0
    while placed>=0 and placed<len(blanks):
        pos        = blanks[placed]
        used      -= cover[board[pos],pos]
        board[pos] = next((n for n in range(board[pos]+1,side+1) if not cover[n,pos]&used),0)
        used      |= cover[board[pos],pos]
        placed    += 1 if board[pos] else -1
        if placed == len(blanks):
            solution = [board[r:r+side] for r in range(0,side*side,side)]
            yield solution
            placed -= 1

In [4]:
from IPython.display import display, HTML

def sudoku_to_html(board):
    """
    Generates an HTML table representation of a Sudoku board with thick edges for subgrids.

    Args:
        board (list of list of int): 2D list representing the Sudoku board.
                                     Empty cells should be represented by 0 or '.'.

    Returns:
        str: HTML string for the Sudoku board.
    """
    html = """
    <style>
        table { border-collapse: collapse; font-family: Arial, sans-serif; }
        td { border: 1px solid black; height: 40px; width: 40px; text-align: center; font-size: 18px; }
        .thick-border-top { border-top: 3px solid black; }
        .thick-border-left { border-left: 3px solid black; }
    </style>
    <table>
    """

    for i, row in enumerate(board):
        html += "<tr>"
        for j, cell in enumerate(row):
            # Apply thicker borders for subgrid boundaries
            classes = []
            if i % 3 == 0 and i != 0:
                classes.append("thick-border-top")
            if j % 3 == 0 and j != 0:
                classes.append("thick-border-left")
            
            class_attr = f'class="{" ".join(classes)}"' if classes else ""
            html += f"<td {class_attr}>{cell if cell != 0 else ''}</td>"
        html += "</tr>"

    html += "</table>"
    return html

In [5]:
# Generate and display the Sudoku board as HTML

display(HTML(sudoku_to_html(board)))

display(HTML(sudoku_to_html(solution)))


2,,,7,,,,,
5,,,,,,,1,
,8,,3,,,,9,
,6,,2,9,7,,,
,,,,,,,,
,,8,,,,,,
,,,9,,5,,,
,,,1,7,,,,
8,,5,4,3,,,2,


2,3,1,7,5,9,6,4,8
5,7,9,8,6,4,2,1,3
6,8,4,3,2,1,5,9,7
1,6,3,2,9,7,4,8,5
9,2,7,5,4,8,1,3,6
4,5,8,6,1,3,9,7,2
7,1,2,9,8,5,3,6,4
3,4,6,1,7,2,8,5,9
8,9,5,4,3,6,7,2,1


In [6]:
board

array([[2, 0, 0, 7, 0, 0, 0, 0, 0],
       [5, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 8, 0, 3, 0, 0, 0, 9, 0],
       [0, 6, 0, 2, 9, 7, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 8, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 9, 0, 5, 0, 0, 0],
       [0, 0, 0, 1, 7, 0, 0, 0, 0],
       [8, 0, 5, 4, 3, 0, 0, 2, 0]])

1. Represent the Sudoku problem in CSP by identifying the variable, domain, and constraint.

In [48]:
def domain_cleaner(variables_dict, variable):
    """
    Cleans the domain of the variable by removing values that are already assigned to other variables.

    Args:
        variables_dict (dict): The CSP variables with their domains and current assignments.
        variable (tuple): The variable (row, col) whose domain needs to be cleaned.

    Returns:
        list: The cleaned domain for the variable.
    """
    # row = variables_dict[variable]["row"]
    # col = variables_dict[variable]["col"]
    
    # block_row = row // 3
    # block_col = col // 3
    # # Get the current value of the variable
    # current_value = variables_dict[variable]["value"]
    
    # Initialize a set to store values that are already assigned in the same row, column, or block
    # assigned_values = set()
    neighbour_values = [variables_dict[neighbour]["value"] for neighbour in variables_dict[variable]["Neighbours"] if variables_dict[neighbour]["value"] > 0]
    # for neighbour in variables_dict[variable]["Neighbours"]:
    #     if variables_dict[neighbour]["value"]>0:
    # for r in range(9):
    #     for c in range(9):
    #         if (r, c) != variable and variables_dict[(r, c)]["Filled"]:
    #             # Prune domains of variables in the same row, column, or block, that are filled
    #             if r == row or c == col or (r // 3 == block_row and c // 3 == block_col):
    #                 assigned_values.add(variables_dict[(r, col)]["value"])

    cleaned_domain = [val for val in variables_dict[variable]["domain"] if val not in neighbour_values]  
    return cleaned_domain
    # cleaned_domain = [val for val in variables_dict[variable]["domain"] if val not in assigned_values]
    # variables_dict[variable]["domain"] = cleaned_domain
    # return variables_dict

In [28]:
def constraints_check(variable,variables_dict, test_value):
    """
    Checks if the test value satisfies all constraints.

    Args:
        assignment (dict): A dictionary mapping variables (row, col) to their assigned values.
        variables_dict (dict): The CSP variables with their domains.
        valriable (tuple): position of the cell on the board.

    Returns:
        bool: True if all constraints are satisfied, False otherwise.
    """
    # check constraints
    neighbour_values = [variables_dict[neighbour]["value"] for neighbour in variables_dict[variable]["Neighbours"] if variables_dict[neighbour]["value"] > 0]
    if test_value in neighbour_values:
        return False
    else:
        # If all constraints are satisfied, return True
        return True

In [29]:
def represent_sudoku_csp(board):
    """
    Represents a Sudoku board as a Constraint Satisfaction Problem (CSP) with detailed variable and constraint structures.

    Args:
        board (list of list of int): A 9x9 2D list representing the Sudoku board.
                                     Empty cells are represented by 0.

    Returns:
        dict: A dictionary where each key is a tuple (row, col) representing a cell, and the value is another dictionary containing:
            - 'row' (int): The row index of the cell.
            - 'col' (int): The column index of the cell.
            - 'value' (int or None): The value assigned to the cell, or None if the cell is empty.
            - 'domain' (list of int): The possible values that can be assigned to the cell.
                                      For pre-filled cells, this is a single-value list.
            - 'constraints' (dict): A dictionary of constraint functions for the cell, including:
                - 'row': A function to enforce the row constraint.
                - 'col': A function to enforce the column constraint.
                - 'box': A function to enforce the 3x3 subgrid constraint.

    Notes:
        - This function provides a more detailed representation of the CSP, where each variable (cell) is associated with its constraints and domain.
        - The constraints ensure that no number is repeated in the same row, column, or 3x3 subgrid.
        - This representation can be used with CSP solvers to solve the Sudoku puzzle.
    """
    side = len(board)
    variables = {}

    for row in range(side):
        for col in range(side):
            # Neighbours = [] [ (r,c)  ]
            # for r in range(side):
            #     for c in range(c) if row == r or col == c or (row // 3 == r // 3 and col // 3 == c // 3)
            #      # list of neighbours
            #     "Filled_Neighbours": [(r,c) for r in range(side) for c in range(c) if (row == r or col == c or (row // 3 == r // 3 and col // 3 == c // 3)) and board[row][col]>0], # list of neighbours that are already filled
            
            variable = {
                'row': row,
                'col': col,
                'value': board[row][col], # initial value, zero represents not filled yet
                'domain': [board[row][col]] if board[row][col]>0 else list(range(1, 10)), # initial domains
                # 'Filled': True if board[row][col]>0 else False, # value given by the puzzle
                "Neighbours": [ (r,c) for r in range(side) for c in range(side) if row == r or col == c or (row // 3 == r // 3 and col // 3 == c // 3) ], # list of neighbours
                "Filled_Neighbours": [(r,c) for r in range(side) for c in range(side) if (row == r or col == c or (row // 3 == r // 3 and col // 3 == c // 3)) and board[r][c]>0], # list of neighbours that are already filled
                # 'constraints': {
                #     'row': row_constraint, # function that returns true if the value does not exist in the row
                #     'col': col_constraint, # function that returns true if the value does not exist in the column
                #     'box': block_constraint # function that returns true if the value does not exist in the block
                # }
            }
        
            variables[(row, col)] = variable
    return variables

In [51]:
variables_dict = represent_sudoku_csp(board)
for variable in variables_dict:
    # Clean the domain of the variable
    variables_dict[variable]["domain"] = domain_cleaner(variables_dict, variable)
key = list(variables_dict.keys())[1]
print(key)
pprint(variables_dict[key])
# CSP representation of the first cell of the Sudoku board

(0, 1)
{'Filled_Neighbours': [(0, 0), (0, 3), (1, 0), (2, 1), (3, 1)],
 'Neighbours': [(0, 0),
                (0, 1),
                (0, 2),
                (0, 3),
                (0, 4),
                (0, 5),
                (0, 6),
                (0, 7),
                (0, 8),
                (1, 0),
                (1, 1),
                (1, 2),
                (2, 0),
                (2, 1),
                (2, 2),
                (3, 1),
                (4, 1),
                (5, 1),
                (6, 1),
                (7, 1),
                (8, 1)],
 'col': 1,
 'domain': [1, 3, 4, 9],
 'row': 0,
 'value': 0}


2. Implement the problem using backtracking search, what is avg. time taken by the algorithm for 10 runs.

In [ ]:
def new_backtracking_search(variables_dict):

    Unfilled = [ v for v in variables_dict if variables_dict[v]["value"] ==0 ]
    
    # If all variables are assigned, return the assignment
    if len(Unfilled) == 0:
        return variables_dict
    
    variable = Unfilled[0]

    # Try each value in the domain of the variable
    for test_domain_value in variables_dict[variable]['domain']:

        # Create a new variables dictionary with a test value for the  domain 
        
        # copy_variables_dict[variable]["domain"] = test_domain_value

        # Check if a test value for the domain is valid
        constraints_check_flag = constraints_check(
            variable = variable,
            variables_dict = variables_dict,
            test_value = test_domain_value)
        
        if constraints_check_flag:
            copy_variables_dict = copy.deepcopy(variables_dict)
            # Recursively solve the CSP with the new assignment
            copy_variables_dict[variable]["value"] = test_domain_value
            # copy_variables_dict[variable]["Filled"] = True

            result = new_backtracking_search(variables_dict = copy_variables_dict)
            if result is not None:
                return result
    # If no value leads to a solution, return None
    return None


In [63]:
%%time
Solved_board_dict = new_backtracking_search(variables_dict)
solved_board = np.zeros((9, 9), dtype=int)
for row in range(side):
    for col in range(side):
        solved_board[row][col] = Solved_board_dict[(row, col)]["value"]

CPU times: user 4.95 s, sys: 0 ns, total: 4.95 s
Wall time: 4.95 s


The solved board may be different from the original solution board,  
as the input board can have multiple solutions depending on the numbers removed

In [64]:
display(HTML(sudoku_to_html(solved_board)))
display(HTML(sudoku_to_html(solution)))

2,1,3,7,4,9,5,6,8
5,4,9,6,2,8,3,1,7
6,8,7,3,5,1,2,9,4
1,6,4,2,9,7,8,5,3
3,5,2,8,1,4,6,7,9
7,9,8,5,6,3,1,4,2
4,2,1,9,8,5,7,3,6
9,3,6,1,7,2,4,8,5
8,7,5,4,3,6,9,2,1


2,3,1,7,5,9,6,4,8
5,7,9,8,6,4,2,1,3
6,8,4,3,2,1,5,9,7
1,6,3,2,9,7,4,8,5
9,2,7,5,4,8,1,3,6
4,5,8,6,1,3,9,7,2
7,1,2,9,8,5,3,6,4
3,4,6,1,7,2,8,5,9
8,9,5,4,3,6,7,2,1


3. Analyse how different fault finding algorithms such as Forward Checking, Arc consistency improve the computational time of backtracking search?

In [68]:
def forward_checking_arc_consistency(variables_dict, variable):
    """
    Performs forward checking by pruning the domains of unassigned variables.

    Args:
        variables_dict (dict): The CSP variables with their domains and current assignments.
        variable (tuple): The variable (row, col) that was just assigned a value.
        test_domain_value (int): The value assigned to the variable.

    Returns:
        bool: True if no domain becomes empty after pruning, False otherwise.
    """
    copy_variables_dict_forward_checked = copy.deepcopy(variables_dict)
    test_domain_value = copy_variables_dict_forward_checked[variable]["value"]
    for neighbour in copy_variables_dict_forward_checked[variable]["Neighbours"]:
        # Prune domains of variables in the neighbourhood that are not filled
        if copy_variables_dict_forward_checked[neighbour]["value"] == 0:
            if test_domain_value in copy_variables_dict_forward_checked[neighbour]["domain"]:
                copy_variables_dict_forward_checked[neighbour]["domain"].remove(test_domain_value)
                # If domain becomes empty, return False
                if len(copy_variables_dict_forward_checked[neighbour]["domain"]) == 0:
                    return False

    return copy_variables_dict_forward_checked

In [69]:
%%time

def forward_checking_backtracking_search(variables_dict):
    """
    Solves the CSP using backtracking search with Forward Checking.

    Args:
        variables_dict (dict): The CSP variables with their domains and current assignments.

    Returns:
        dict or None: A complete assignment if a solution is found, or None if no solution exists.
    """
    # Find unfilled variables
    unfilled = [v for v in variables_dict if variables_dict[v]["value"]==0]

    # If all variables are assigned, return the assignment
    if not unfilled:
        return variables_dict

    # Select the first unfilled variable
    variable = unfilled[0]

    # Try each value in the domain of the variable
    for test_domain_value in variables_dict[variable]["domain"]:

        # Create a new variables dictionary with a test value for the  domain 
        
        # Constraints check will not be required as:
        # 1. the domains are cleaned
        # 2. forward checking prempts conflicts
        
        copy_variables_dict_test_domain = copy.deepcopy(variables_dict)
        
        #set the test value to the variable
        copy_variables_dict_test_domain[variable]["value"] = test_domain_value

        # Use Forward Checking to prune the domains of unassigned variables
        # and check if any domain becomes empty 
        copy_variables_dict_forward_checked = forward_checking_arc_consistency(copy_variables_dict_test_domain, variable)
        if copy_variables_dict_forward_checked:
            # Recursively solve the CSP with the new assignment
            result = forward_checking_backtracking_search(copy_variables_dict_forward_checked)
            if result is not None:
                return result
                
    # If no value leads to a solution, return None
    return None

CPU times: user 6 μs, sys: 0 ns, total: 6 μs
Wall time: 8.82 μs


In [70]:
%%time
solved_board_dict = forward_checking_backtracking_search(variables_dict)
# Convert the solved board dictionary back to a 2D array
if solved_board_dict:
    solved_board = np.zeros((9, 9), dtype=int)
    for row in range(9):
        for col in range(9):
            solved_board[row][col] = solved_board_dict[(row, col)]["value"]

    # Display the solved board
    html_output = sudoku_to_html(solved_board)
    display(HTML(html_output))
else:
    print("No solution exists.")


2,1,3,7,4,9,5,6,8
5,4,9,6,2,8,3,1,7
6,8,7,3,5,1,2,9,4
1,6,4,2,9,7,8,5,3
3,5,2,8,1,4,6,7,9
7,9,8,5,6,3,1,4,2
4,2,1,9,8,5,7,3,6
9,3,6,1,7,2,4,8,5
8,7,5,4,3,6,9,2,1


CPU times: user 4.94 s, sys: 9.97 ms, total: 4.95 s
Wall time: 4.95 s


In [42]:
sys.exi()

AttributeError: module 'sys' has no attribute 'exi'

In [36]:
%%time
solved_board_dict = forward_checking_backtracking_search(variables_dict)
# Convert the solved board dictionary back to a 2D array
if solved_board_dict:
    solved_board = np.zeros((9, 9), dtype=int)
    for row in range(9):
        for col in range(9):
            solved_board[row][col] = solved_board_dict[(row, col)]["value"]

    # Display the solved board
    html_output = sudoku_to_html(solved_board)
    display(HTML(html_output))
else:
    print("No solution exists.")

No solution exists.
CPU times: user 9.11 s, sys: 0 ns, total: 9.11 s
Wall time: 9.1 s


d.	Analyse how different Heuristics MRV (Minimum Remaining Values), Degree heuristic, Least Constraining Value affect the  computational time of backtracking search?

In [ ]:
def select_variable_with_heuristics(variables_dict):
    """
    Selects the next variable to assign using MRV and Degree Heuristic.

    Args:
        variables_dict (dict): The CSP variables with their domains and current assignments.

    Returns:
        tuple: The selected variable (row, col).
    """
    # Filter unfilled variables
    unfilled = [v for v in variables_dict if not variables_dict[v]["Filled"]]

    # Apply MRV (Minimum Remaining Values)
    mrv = min(unfilled, key=lambda v: len(variables_dict[v]["domain"]))

    # If there are ties, apply Degree Heuristic
    tied_variables = [v for v in unfilled if len(variables_dict[v]["domain"]) == len(variables_dict[mrv]["domain"])]
    if len(tied_variables) > 1:
        # Degree Heuristic: Select the variable involved in the most constraints
        mrv = max(tied_variables, key=lambda v: sum(
            not variables_dict[neighbor]["Filled"] for neighbor in variables_dict if is_neighbor(v, neighbor)
        ))

    return mrv

In [ ]:
def order_domain_values(variable, variables_dict):
    """
    Orders the domain values of a variable using the Least Constraining Value (LCV) heuristic.

    Args:
        variable (tuple): The variable (row, col) to order domain values for.
        variables_dict (dict): The CSP variables with their domains and current assignments.

    Returns:
        list: The ordered list of domain values.
    """
    def count_conflicts(value):
        # Count how many variables would be affected by assigning this value
        return sum(
            value in variables_dict[neighbor]["domain"]
            for neighbor in variables_dict if is_neighbor(variable, neighbor) and not variables_dict[neighbor]["Filled"]
        )

    # Sort domain values by the number of conflicts (ascending)
    return sorted(variables_dict[variable]["domain"], key=count_conflicts)

In [ ]:
def is_neighbor(var1, var2):
    """
    Checks if two variables are neighbors (i.e., in the same row, column, or block).

    Args:
        var1 (tuple): The first variable (row, col).
        var2 (tuple): The second variable (row, col).

    Returns:
        bool: True if the variables are neighbors, False otherwise.
    """
    row1, col1 = var1
    row2, col2 = var2
    return row1 == row2 or col1 == col2 or (row1 // 3 == row2 // 3 and col1 // 3 == col2 // 3)

In [ ]:
def backtracking_search_with_heuristics(variables_dict):
    """
    Solves the CSP using backtracking search with MRV, Degree Heuristic, and LCV.

    Args:
        variables_dict (dict): The CSP variables with their domains and current assignments.

    Returns:
        dict or None: A complete assignment if a solution is found, or None if no solution exists.
    """
    # Check if all variables are assigned
    if all(variables_dict[v]["Filled"] for v in variables_dict):
        return variables_dict

    # Select the next variable to assign using MRV and Degree Heuristic
    variable = select_variable_with_heuristics(variables_dict)

    # Order domain values using LCV
    for value in order_domain_values(variable, variables_dict):
        # Check if the value satisfies all constraints
        if constraints_check(variable, variables_dict, value):
            # Assign the value to the variable
            variables_dict[variable]["value"] = value
            variables_dict[variable]["Filled"] = True

            # Recursively solve the CSP
            result = backtracking_search_with_heuristics(variables_dict)
            if result is not None:
                return result

            # Undo the assignment (backtrack)
            variables_dict[variable]["value"] = 0
            variables_dict[variable]["Filled"] = False

    # If no value leads to a solution, return None
    return None

In [ ]:
%%time
# Example usage
# variables_dict = represent_sudoku_csp(board)
solved_board_dict = backtracking_search_with_heuristics(variables_dict)

# Convert the solved board dictionary back to a 2D array
if solved_board_dict:
    solved_board = np.zeros((9, 9), dtype=int)
    for row in range(9):
        for col in range(9):
            solved_board[row][col] = solved_board_dict[(row, col)]["value"]

    # Display the solved board
    html_output = sudoku_to_html(solved_board)
    display(HTML(html_output))
else:
    print("No solution exists.")

In [ ]:

solved_board = np.zeros((9, 9), dtype=int)
for row in range(9):
    for col in range(9):
        solved_board[row][col] = variables_dict[(row, col)]["value"]

# Display the solved board
html_output = sudoku_to_html(solved_board)
display(HTML(html_output))

In [ ]:
sys.exit()

In [ ]:
def arc_consistency(variables_dict):
    """
    Enforces arc consistency by pruning the domains of variables.

    Args:
        variables_dict (dict): The CSP variables with their domains and current assignments.

    Returns:
        bool: True if arc consistency is maintained, False if any domain becomes empty.
    """
    queue = [(x, y) for x in variables_dict for y in variables_dict if x != y]

    while queue:
        (x, y) = queue.pop(0)
        if revise(variables_dict, x, y):
            if not variables_dict[x]["domain"]:
                return False
            for z in variables_dict:
                if z != x and z != y:
                    queue.append((z, x))
    return True


In [ ]:
def revise(variables_dict, x, y):
    """
    Revises the domain of variable x to ensure consistency with variable y.

    Args:
        variables_dict (dict): The CSP variables with their domains and current assignments.
        x (tuple): The variable to revise.
        y (tuple): The variable to check consistency against.

    Returns:
        bool: True if the domain of x was revised, False otherwise.
    """
    revised = False
    for value in variables_dict[x]["domain"][:]:
        if not any(is_consistent(value, other_value, x, y, variables_dict) for other_value in variables_dict[y]["domain"]):
            variables_dict[x]["domain"].remove(value)
            revised = True
    return revised

In [ ]:
def is_consistent(value_x, value_y, x, y, variables_dict):
    """
    Checks if assigning value_x to x is consistent with assigning value_y to y.

    Args:
        value_x (int): The value assigned to variable x.
        value_y (int): The value assigned to variable y.
        x (tuple): The variable x.
        y (tuple): The variable y.
        variables_dict (dict): The CSP variables with their domains and current assignments.

    Returns:
        bool: True if consistent, False otherwise.
    """
    if value_x == value_y:
        row_x, col_x = variables_dict[x]["row"], variables_dict[x]["col"]
        row_y, col_y = variables_dict[y]["row"], variables_dict[y]["col"]
        # Check if x and y are in the same row, column, or block
        if row_x == row_y or col_x == col_y or (row_x // 3 == row_y // 3 and col_x // 3 == col_y // 3):
            return False
    return True

In [ ]:
def new_backtracking_search_with_arc_consistency(variables_dict):
    """
    Solves the CSP using backtracking search with Arc Consistency.

    Args:
        variables_dict (dict): The CSP variables with their domains and current assignments.

    Returns:
        dict or None: A complete assignment if a solution is found, or None if no solution exists.
    """
    # Find unfilled variables
    unfilled = [v for v in variables_dict if not variables_dict[v]["Filled"]]

    # If all variables are assigned, return the assignment
    if not unfilled:
        return variables_dict

    # Select the first unfilled variable
    variable = unfilled[0]

    for test_domain_value in variables_dict[variable]["domain"]:

        # Check if a test value for the domain is valid
        constraints_check_flag = constraints_check(
            variable = variable,
            variables_dict = variables_dict,
            test_value = test_domain_value)
        
        if constraints_check_flag:
            copy_variables_dict_test_domain = copy.deepcopy(variables_dict)
            # Recursively solve the CSP with the new assignment
            copy_variables_dict_test_domain[variable]["value"] = test_domain_value
            copy_variables_dict_test_domain[variable]["Filled"] = True

            # Enforce arc consistency
            if arc_consistency(copy_variables_dict_test_domain):
                # Recursively solve the CSP with the new assignment
                result = new_backtracking_search_with_arc_consistency(copy_variables_dict_test_domain)
                if result is not None:
                    return result

    # If no value leads to a solution, return None
    return None

In [ ]:
%%time
# # Example usage
# variables_dict = represent_sudoku_csp(board)
solved_board_dict = new_backtracking_search_with_arc_consistency(variables_dict)

# Convert the solved board dictionary back to a 2D array
if solved_board_dict:
    solved_board = np.zeros((9, 9), dtype=int)
    for row in range(9):
        for col in range(9):
            solved_board[row][col] = solved_board_dict[(row, col)]["value"]

    # Display the solved board
    html_output = sudoku_to_html(solved_board)
    display(HTML(html_output))
else:
    print("No solution exists.")

Explanation:  

1. Arc Consistency:
    - The arc_consistency function ensures that for every variable, all values in its domain are consistent with the domains of other variables.
    - It uses a queue of variable pairs and iteratively revises the domains to maintain consistency.
2. Revise Function:
    - The revise function removes values from the domain of a variable if they are inconsistent with the domain of another variable.
3. Backtracking with Arc Consistency:
    - The new_backtracking_search_with_arc_consistency function integrates arc consistency into the backtracking search.
    - Before assigning a value, it enforces arc consistency to prune the search space.
4. Efficiency:
    - Arc Consistency reduces the search space by eliminating inconsistent values early, improving the computational efficiency of the backtracking search.

In [ ]:
sys.exit()

In [ ]:
def forward_checking(variables_dict, variable, value):
    """
    Performs forward checking by pruning the domains of unassigned variables.

    Args:
        variables_dict (dict): The CSP variables with their domains and current assignments.
        variable (tuple): The variable (row, col) that was just assigned a value.
        value (int): The value assigned to the variable.

    Returns:
        bool: True if no domain becomes empty after pruning, False otherwise.
    """
    row, col = variables_dict[variable]["row"], variables_dict[variable]["col"]

    # Check row, column, and block constraints
    block_row, block_col = row // 3, col // 3
    for r in range(9):
        for c in range(9):
            if (r, c) != variable and not variables_dict[(r, c)]["Filled"]:
                # Prune domains of variables in the same row, column, or block
                if r == row or c == col or (r // 3 == block_row and c // 3 == block_col):
                    if value in variables_dict[(r, c)]["domain"]:
                        variables_dict[(r, c)]["domain"].remove(value)
                        # If domain becomes empty, return False
                        if not variables_dict[(r, c)]["domain"]:
                            return False
    return True


def new_backtracking_search(variables_dict):
    """
    Solves the CSP using backtracking search with Forward Checking.

    Args:
        variables_dict (dict): The CSP variables with their domains and current assignments.

    Returns:
        dict or None: A complete assignment if a solution is found, or None if no solution exists.
    """
    # Find unfilled variables
    unfilled = [v for v in variables_dict if not variables_dict[v]["Filled"]]

    # If all variables are assigned, return the assignment
    if not unfilled:
        return variables_dict

    # Select the first unfilled variable
    variable = unfilled[0]

    # Try each value in the domain of the variable
    for value in variables_dict[variable]["domain"]:
        # Create a deep copy of the variables dictionary
        copy_variables_dict = copy.deepcopy(variables_dict)

        # Assign the value to the variable
        copy_variables_dict[variable]["value"] = value
        copy_variables_dict[variable]["Filled"] = True

        # Perform forward checking
        if forward_checking(copy_variables_dict, variable, value):
            # Recursively solve the CSP with the new assignment
            result = new_backtracking_search(copy_variables_dict)
            if result is not None:
                return result

    # If no value leads to a solution, return None
    return None


# Example usage
variables_dict = represent_sudoku_csp(board)
solved_board_dict = new_backtracking_search(variables_dict)

# Convert the solved board dictionary back to a 2D array
if solved_board_dict:
    solved_board = np.zeros((9, 9), dtype=int)
    for row in range(9):
        for col in range(9):
            solved_board[row][col] = solved_board_dict[(row, col)]["value"]

    # Display the solved board
    html_output = sudoku_to_html(solved_board)
    display(HTML(html_output))
else:
    print("No solution exists.")

In [ ]:
def is_assignment_valid(assignment, variables, constraints):
    """
    Checks if the current assignment satisfies all constraints.

    Args:
        assignment (dict): A dictionary mapping variables (row, col) to their assigned values.
        variables (dict): The CSP variables with their domains and constraints.
        constraints (list): A list of constraint functions.

    Returns:
        bool: True if all constraints are satisfied, False otherwise.
    """
    for constraint in constraints:
        if not constraint(assignment):
            return False
    return True

In [ ]:
def backtracking_search(variables, domains, constraints, assignment={}):
    """
    Solves the CSP using backtracking search.
    
    Steps:
    1. Selects an unassigned variable and tries each value in its domain.
    2. Recursively solves the CSP with the new assignment.
    3. Backtracks if no value leads to a solution.

    Args:
        variables (list): List of variables (row, col) in the CSP.
        domains (dict): Dictionary mapping variables to their possible values.
        constraints (list): List of constraint functions.
        assignment (dict): Current assignment of values to variables.

    Returns:
        dict or None: A complete assignment if a solution is found, or None if no solution exists.
    """
    # If all variables are assigned, return the assignment
    if len(assignment) == len(variables):
        return assignment

    # Select an unassigned variable
    unassigned = [v for v in variables if v not in assignment]
    var = unassigned[0]

    # Try each value in the domain of the variable
    for value in domains[var]:
        # Create a new assignment with the current value
        new_assignment = assignment.copy()
        new_assignment[var] = value

        # Check if the new assignment is valid
        if is_assignment_valid(new_assignment, variables, constraints):
            # Recursively solve the CSP with the new assignment
            result = backtracking_search(variables, domains, constraints, new_assignment)
            if result is not None:
                return result

    # If no value leads to a solution, return None
    return None

In [ ]:
variables_dict = represent_sudoku_csp(board)
variables = list(variables_dict.keys())
domains = {var: variables_dict[var]['domain'] for var in variables}
constraints = [
    lambda assignment: row_constraint(assignment, row) for row in range(9)
] + [
    lambda assignment: col_constraint(assignment, col) for col in range(9)
] + [
    lambda assignment: block_constraint(assignment, block_row, block_col)
    for block_row in range(3) for block_col in range(3)
]

solution = backtracking_search(variables, domains, constraints)
if solution:
    print("Solution found:")
    pprint(solution)
else:
    print("No solution exists.")